# Pipelines de operaciones

Agrupar múltiples operaciones en un mismo objeto:

* Imputar nulos
* Codificación de categóricos
* Escalado de datos
* Modelado

In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor


## Carga de datos

In [39]:
df = sns.load_dataset('penguins')
df.isnull().sum()

species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
dtype: int64

## Nulos en columna salida (y)

In [40]:
# Imputar nulos en la columna de salida (y) que es body_mass_g
df['body_mass_g'] = SimpleImputer(missing_values=np.nan, strategy='median').fit_transform(df[['body_mass_g']])

# borrar en lugar de imputar
# df = df.dropna('body_mass_g')

## Preparar X y

In [45]:
X = df.drop('body_mass_g', axis=1)
y = df['body_mass_g'] # no tiene nulos

## Pipeline columnas numéricas

In [46]:
pipeline_numeric = Pipeline([
    ('impute_median', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])
numeric_col_names = X.select_dtypes(include=np.number).columns.to_list()
numeric_col_names

['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']

## Pipeline de columnas categóricas

In [47]:
pipeline_categorical = Pipeline([
    ('impute_mode', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='first', sparse=False))
])
categorical_col_names = X.select_dtypes(include='object').columns.to_list()
categorical_col_names

['species', 'island', 'sex']

## Pipeline numéricas + categóricas

In [49]:
preprocessor = ColumnTransformer([
    ('numerical', pipeline_numeric, numeric_col_names),
    ('categorical', pipeline_categorical, categorical_col_names)
])
preprocessor

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('impute_median',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', MinMaxScaler())]),
                                 ['bill_length_mm', 'bill_depth_mm',
                                  'flipper_length_mm']),
                                ('categorical',
                                 Pipeline(steps=[('impute_mode',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False))]),
                                 ['species', 'island', 'sex'])])

## Pipeline numéricas + categóricas + modelado

In [50]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('linear_regression', LinearRegression()) # se puede cambiar el algoritmo
    # ('knn', KNeighborsRegressor())
])
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('impute_median',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['bill_length_mm',
                                                   'bill_depth_mm',
                                                   'flipper_length_mm']),
                                                 ('categorical',
                                                  Pipeline(steps=[('impute_mode',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse=False))]),
                                                  ['species', 'island',
                                                   'sex'])])),
                ('linear_regression', LinearRegression())])

## Ejecución pipeline

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

mean_squared_error(y_test, y_pred, squared=False)


c:\Users\alanj\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


318.7971714438644

## Guardar pipeline

Se usa joblib para guardar modelos o pipelines.

Serializa un modelo o pipeline (objeto Python) a una secuencia de bytes que se almacenan en un archivo

* joblib.dump()
* joblib.load()

In [59]:
import joblib

# pickle .pkl o .joblib
joblib.dump(pipeline, 'pipeline.pkl') # más genérico
joblib.dump(pipeline, 'pipeline.joblib') # más eficiente con muchos datos

['pipeline.joblib']

## Cargar pipeline

In [58]:
pipeline = joblib.load('pipeline.pkl')
pipeline.predict(X_test)[:10]

array([4006.44704878, 3446.65045285, 4652.08396283, 3267.32367752,
       4623.83146849, 5152.74949404, 5560.00223302, 4544.23407991,
       3655.96209525, 4644.37891153])

In [60]:
pipeline = joblib.load('pipeline.joblib')
pipeline.predict(X_test)[:10]

array([4006.44704878, 3446.65045285, 4652.08396283, 3267.32367752,
       4623.83146849, 5152.74949404, 5560.00223302, 4544.23407991,
       3655.96209525, 4644.37891153])

## Crear transformador personalizado (avanzado)

En aquellos casos que no exista un transformador en ScikitLearn para hacer la transformación que necesitemos sobre los datos, por ejemplo, borrar valores outliers, será necesario crear un transformador personalizado.

Se crean mediante una clase que hereda de BaseEstimator y TransformerMixin.

En el método ``fit`` se calculan los parámetros necesarios para la transformación de los datos X. Por ejemplo calcular el IQR y los límites inferior y superior para saber cómo borrar los outliers.

En el método ``transform`` se utilizan los parámetros calculados en fit para transformar los datos. Por ejemplo borrar los outliers en función de los límites inferior y superior calculados en fit.

In [79]:
from sklearn.base import BaseEstimator, TransformerMixin

class Debugger(BaseEstimator, TransformerMixin):
    def __init__(self, message):
        self.message = message
    
    def fit(self, X, y=None):
        # fit se utiliza para calcular los parámetros necesarios 
        # que se deben aprender de los datos y se guardan con self
        print('Hola desde fit')
        self.param1 = 'hola'
        return self # devuelve el propio objeto porque más adelante se necesitará ejecutar transform


    def transform(self, X, y=None):
        # transform transforma los datos basándose en los parámetros aprendidos en fit
        print('Hola desde transform')
        return X # devuelve los datos transformados

## Usar transformador personalizado

In [81]:

pipeline = Pipeline([
    ('debugger1', Debugger('Hola')),
    ('preprocessor', preprocessor),
    ('debugger2', Debugger('Hola')),
    ('linear_regression', LinearRegression()) # se puede cambiar el algoritmo
    # ('knn', KNeighborsRegressor())
])
pipeline.fit(X_train, y_train)
# pipeline.predict(X_test)

Hola desde fit
Hola desde transform
Hola desde fit
Hola desde transform


c:\Users\alanj\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('debugger1', Debugger(message='Hola')),
                ('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('impute_median',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['bill_length_mm',
                                                   'bill_depth_mm',
                                                   'flipper_length_mm']),
                                                 ('categorical',
                                                  Pipeline(steps=[('impute_mode',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse=False))]),
                                                  ['species', 'island',
                                                   'sex'])])),
                ('debugger2', Debugger(message='Hola')),
                ('linear_regression', LinearRegression())])